In [1]:
# Script to Load Date Dimesion

In [2]:
# Import required libraries
import sys
from lib.spark_session import get_spark_session
from lib.utils import date_data, get_string_cols
from lib.job_control import insert_log, get_max_timestamp
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp, expr, to_date, date_format
from datetime import datetime
from delta import DeltaTable

In [5]:
# JOB Parameters
rundate = "20220101"
schema_name = "edw"
table_name = "dim_date"
table_full_name = f"{schema_name}.{table_name}"
staging_table_full_name = "edw_stg.dim_date_stg"

In [6]:
# Generate Spark Session
spark: SparkSession = get_spark_session(f"Dimension load - {table_full_name}")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

SPARK_APP: Spark UI - http://46346aae9d54:4040


In [7]:
# Spark Configs
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [8]:
# Read data from Staging
df_stg = spark \
    .read \
    .table(staging_table_full_name)

print("SPARK_APP: Staging Data Count - " + str(df_stg.count()))
print("SPARK_APP: Printing Staging Schema --")
df_stg.printSchema()

SPARK_APP: Staging Data Count - 729
SPARK_APP: Printing Staging Schema --
root
 |-- date: date (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- insert_dt: timestamp (nullable = true)
 |-- update_dt: timestamp (nullable = true)



In [9]:
# Generate SURROGATE KEYs
df_dim_temp = df_stg.withColumn("row_wid", date_format("date", "yyyyMMdd"))

print("SPARK_APP: Dim Temp Data Count - " + str(df_dim_temp.count()))
print("SPARK_APP: Printing Dim Temp Schema --")
df_dim_temp.printSchema()

SPARK_APP: Dim Temp Data Count - 729
SPARK_APP: Printing Dim Temp Schema --
root
 |-- date: date (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- insert_dt: timestamp (nullable = true)
 |-- update_dt: timestamp (nullable = true)
 |-- row_wid: string (nullable = true)



In [20]:
# Get the delta table for Upserts (SCD1)
dt_dim = DeltaTable.forName(spark, table_full_name)

# Create the UPSERT logic
dt_dim.alias("dim_date") \
    .merge(df_dim_temp.alias("dim_temp"), "dim_date.date = dim_temp.date") \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

In [21]:
# Get the logs from delta table version
dt_dim.history().limit(1).select("version","operationMetrics.executionTimeMs", 
                                 "operationMetrics.numTargetRowsInserted",
                                "operationMetrics.numTargetRowsUpdated",
                                "operationMetrics.numOutputRows").show(1, False)

+-------+---------------+---------------------+--------------------+-------------+
|version|executionTimeMs|numTargetRowsInserted|numTargetRowsUpdated|numOutputRows|
+-------+---------------+---------------------+--------------------+-------------+
|2      |3729           |0                    |729                 |729          |
+-------+---------------+---------------------+--------------------+-------------+



In [18]:
# Generate Symlink manifest for Athena Access
dt = DeltaTable.forName(spark, table_full_name)
dt.generate("symlink_format_manifest")
print("SPARK_APP: Symlink Manifest file generated")

SPARK_APP: Symlink Manifest file generated


In [24]:
spark.stop()